# Comparison of longest common subsequence (LCS) algorithms

This notebooks compares longest common subsequence (LCS) algorithms:

- Brute-force: generate combinations of subseqences and check if they are common subsequences.
- Dynamic programming: take advantage of common subproblems to not evaluate the same subsequence more than once.
- Hirschbger's linear space: a dynamic programming approach that uses significantly less memory.

The comparison measures:

- Runtime efficiency: how long does it take to find the LCS.
- Memory efficiency: how much memory is used to find the LCS.

## Sanity check and initialization

Check that the algorithms work by testing them against controlled input.

In [68]:
import lcs_brute_force
import lcs_dynamic_programming
import lcs_hirschberg
import lcs_test

lcs_test.test()

All tests passed


Set a seed to make pseuudo-random generator generate the same sequence across runs. This mamkes it easier to compare different runs of the algorithms.

In [69]:
import random
random.seed(42)

## Tests

To illustrate a real-life scenario, the code checks if a DNA strand is part of
a larger DNA sequence (see [this for an illustration](https://en.wikipedia.org/wiki/Subsequence#Applications)).

An auxiliary function to generate random DNA sequences.

In [70]:
def random_dna_sequence(length):
    '''Generates a random DNA sequence of the given length.'''
    return ''.join(random.choice('ACTG') for _ in range(length))

The DNA sequences to test on.

In [71]:
dna = random_dna_sequence(10_000)
dna_strand = random_dna_sequence(1_000)

In [72]:
# assert(lcs_brute_force.lcs(dna, dna_strand) == lcs_dynamic_programming.lcs(dna, dna_strand))
# assert(lcs_brute_force.lcs(dna, dna_strand) == lcs_hirschberg.lcs(dna, dna_strand))

In [75]:
%%time
lcs_brute_force.lcs(dna, dna_strand)
print('')


CPU times: user 1.37 ms, sys: 106 µs, total: 1.48 ms
Wall time: 1.5 ms


In [76]:
%%time
lcs_dynamic_programming.lcs(dna, dna_strand)
print('')


CPU times: user 14.4 s, sys: 913 ms, total: 15.3 s
Wall time: 15.5 s


In [77]:
%%time
lcs_hirschberg.lcs(dna, dna_strand)
print('')


CPU times: user 6.43 s, sys: 57 ms, total: 6.49 s
Wall time: 6.56 s


In [83]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [91]:
from collections import defaultdict, namedtuple
from itertools import product

# To saves memory and speed-up the code, the move is stored
# in the upper bits of a cell and the length in the lower bits
MOVE_START = 25 # leaves 24 bits for the length
MOVE_NONE = 0
MOVE_DIAGONAL = 1 << (MOVE_START)
MOVE_UP = 1 << (MOVE_START+1)
MOVE_LEFT = 1 << (MOVE_START+2)
EXCLUDE_LENGTH = (MOVE_DIAGONAL + MOVE_UP + MOVE_LEFT)
EXCLUDE_MOVE = ~EXCLUDE_LENGTH

def _lcs_grid(xs, ys):
    r'''Creates a grid for longest common subsequence calculations.

    Returns a grid where grid[i][j] is a pair (n, move) such that
    - n is the length of the LCS of prefixes xs[:i], ys[:j]
    - move is diagonal, up, left, encoded in the top-most bits

    Example:
      T      A      R      O      T
    A 0|left 1|diag 1|left 1|left 1|left
    R 0|left 1|up   2|diag 2|left 2|left
    T 1|diag 1|left 2|up   2|left 3|diag
    '''

    # A grid of xs columns and ys rows
    # Note that we start with 1, not the traditional 0 because the first
    # column and first row are used as sentinels to avoid special cases
    # in the code (sentinel = length is zero, no move)
    grid = [[0 for i in range(len(xs)+1)] for j in range(len(ys)+1)]

    for i in range(1, len(x)+1):
        x = xs[i-1] # Remember that we use index 0 as a sentinel
        for j in range(1, lenx(ys)+1):
            y = ys[j-1]
            if x == y:
                # A match - move diagonally
                grid[i][j] = (grid[i-1][j-1] & EXCLUDE_MOVE) | MOVE_DIAGONAL
            else:
                left_length = grid[i][j-1] & EXCLUDE_MOVE
                above_length = grid[i-1][j] & EXCLUDE_MOVE
                if left_length < above_length:
                    grid[i][j] = above_length | MOVE_UP
                else:
                    grid[i][j] = left_length | MOVE_LEFT
    return grid


def lcs(xs, ys):
    '''Returns a longest common subsequence of xs, ys.'''
    grid = _lcs_grid(xs, ys)

    # Will accumulate the LCS, in reverse order
    lcs = list()

    # Start that bottom-right corner
    i = len(xs)
    j = len(ys)
    # Follow the moves
    while i > 0 and j > 0:
        move = grid[i][j] & EXCLUDE_LENGTH
        if move == MOVE_DIAGONAL:
            # A match - accumulate and move diagonally (top, left)
            lcs.append(xs[i-1]) # Remeber that 0 is a sentinel
            i -= 1
            j -= 1
        elif move == MOVE_UP:
            i -= 1
        elif move == MOVE_LEFT:
            j -= 1

    lcs.reverse()
    return lcs


IndentationError: expected an indented block (<ipython-input-91-aa7d65ac0196>, line 39)

In [87]:
%lprun -f _lcs_grid lcs(dna, dna_strand)

Timer unit: 1e-06 s

Total time: 169.098 s
File: <ipython-input-84-3fd1e4e23a33>
Function: _lcs_grid at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                           def _lcs_grid(xs, ys):
     6                                               r'''Creates a grid for longest common subsequence calculations.
     7                                           
     8                                               Returns a grid where grid[(j, i)] is a pair (n, move) such that
     9                                               - n is the length of the LCS of prefixes xs[:i], ys[:j]
    10                                               - move is \, ^, <, or e, depending on whether the best move
    11                                               to (j, i) was diagonal, downwards, or rightwards, or None.
    12                                           
    13                                               Example:
    14                 

In [88]:
    grid = [[ 0 for i in range(3)] for j in range(10)]

In [89]:
print(grid)

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
